In [182]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
from scipy.signal import argrelextrema
# from collections import deque
# from matplotlib.lines import Line2D
import pandas_ta as ta
import ccxt
from datetime import datetime, timedelta
# import plotly.graph_objects as go
# import inspect 
import os
# import ccxt_data_fetch
from IPython.display import display, HTML
from tqdm import tqdm
import re

# pd.set_option('display.max_rows', None)
pd.reset_option("display.max_rows")

In [183]:
def calc():
    # data=ccxt_data() # using it will reset all the params to default parms
    # symbol='ETH/USDT'

    global order
    global peaks_df
    global troughs_df    

    if len(data) > 14:
    

        data['ta_rsi'] = ta.rsi(data['Close'], length = 14)

        coin_peak = argrelextrema(data['Close'].values, np.greater, order=order)[0]
        coin_peaks_df = data.iloc[coin_peak]['Close']

        #high instead of close
        # coin_peak = argrelextrema(data['High'].values, np.greater, order=order)[0]
        # coin_peaks_df = data.iloc[coin_peak]['High']
  

        ta_peak = argrelextrema(data['ta_rsi'].values, np.greater, order=order)[0]
        ta_peaks_df = data.iloc[ta_peak]['ta_rsi']
 

        coin_trough = argrelextrema(data['Close'].values, np.less, order=order)[0]
        coin_trough_df = data.iloc[coin_trough]['Close']

        # low instaed of close
        # coin_trough = argrelextrema(data['Low'].values, np.less, order=order)[0]
        # coin_trough_df = data.iloc[coin_trough]['Low']

        ta_trough = argrelextrema(data['ta_rsi'].values, np.less, order=order)[0]
        ta_trough_df = data.iloc[ta_trough]['ta_rsi']

        peaks = [coin_peaks_df, ta_peaks_df]
        peaks_df = pd.concat(peaks, axis=1)
        peaks_df = peaks_df[(peaks_df.Close.notna()) & (peaks_df.ta_rsi.notna())]

        #high instead of close
        # peaks_df = peaks_df[(peaks_df.High.notna()) & (peaks_df.ta_rsi.notna())]


        troughs = [coin_trough_df, ta_trough_df]
        troughs_df = pd.concat(troughs, axis=1)
        troughs_df = troughs_df[(troughs_df.Close.notna()) & (troughs_df.ta_rsi.notna())]

        #low instead of close
        # troughs_df = troughs_df[(troughs_df.Low.notna()) & (troughs_df.ta_rsi.notna())]

        if peaks_df.empty:
            # tof=False
            # print('empty peaks', symbol)
            return False

        elif troughs_df.empty:
            # print('empty troughs', symbol)
            return False
        
        else:
            return peaks_df, troughs_df, True

    else:
        # tof=False
        print('data length less than 14', symbol)
        return False




In [184]:

# peaks_df, troughs_df = calc()

def signal_dates(n=9):
    global sell
    global buy
    sell = []
    buy = []

    # global dates_of_peaks
    dates_of_peaks = []
    
    dates_of_troughs = []

    if calc():

        for i in range(1,n):

            diff = peaks_df.diff(i)
            # diff.dropna(inplace=True) # do not use it
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_peaks.append(sig.index.values)

    
            diff = troughs_df.diff(i)
            # diff.dropna(inplace=True) # do not use it
            sig = diff[~((diff > 0).all(1)) & ~((diff < 0).all(1))]
            dates_of_troughs.append(sig.index.values)

        for list in dates_of_peaks:
            for i in list:
                if str(i) not in sell:
                    # np.datetime to str
                    # maybe pd.to_datetime(i) works better
                    sell.append(str(i))

        for list in dates_of_troughs:
            for i in list:
                if str(i) not in buy:
                    buy.append(str(i))
                    

        sell.sort()
        buy.sort()

        # print(dates_of_peaks)
        # problem : buy ans sell should be list of datetime***********************

        # sell=pd.to_datetime(sell)
        # buy=pd.to_datetime(buy)


        if not buy or not sell:
            print('empty buy or sell')
            return False
        else:
            return sell, buy, True

    
            
    else:
        # tof=False
        # print(' calc() is false ', symbol)
        return False



In [185]:
from binance.client import Client
import datetime as dt
# from functools import cache


client = Client()

# @cache
def get_historical_ohlc_data(symbol, interval='1h'):
  print(symbol)
  
  df = pd.DataFrame(
      client.get_historical_klines(symbol=symbol, start_str="33 day ago UTC", interval=interval))

  df.columns = [
      'open_time', 'Open', 'High', 'Low', 'Close', 'Volume', 'close_time',
      'qav', 'num_trades', 'taker_base_vol', 'taker_quote_vol',
      'is_best_match'
  ]
  df['Time'] = [
      dt.datetime.fromtimestamp(x / 1000) for x in df.open_time
  ]
  df = df[['Time', 'Open', 'High', 'Low', 'Close', 'Volume']]
  df.Close = df.Close.astype(float)
  df.set_index('Time', inplace=True)
#   print(df)
  return df[:-1]

In [186]:
pairs= ['FLUXUSDT', 'DREPUSDT', 'HOTUSDT' ,'ARUSDT', 'SYSUSDT', 'CKBUSDT',\
        'CTXCUSDT', 'DOGEUSDT', 'ETCUSDT','FILUSDT','FIROUSDT', 'LTCUSDT',\
        'KMDUSDT', 'RVNUSDT', 'SCUSDT', 'XLMUSDT',\
        'XMRUSDT', 'XRPUSDT', 'ZECUSDT', 'ZENUSDT', 'DGBUSDT', 'DASHUSDT',\
        'BCHUSDT', 'RNDRUSDT', 'KDAUSDT', 'NKNUSDT']



# candy = []
n=9
order=5

for symbol in pairs:
    data = get_historical_ohlc_data(symbol, interval='1h')
    calc()
    signal_dates(n)
    print(f'buy : \n{buy[-1]} \n{buy[-2]} \n{buy[-3]} \nsell : \n{sell[-1]}')






FLUXUSDT
buy : 
2022-06-27T14:00:00.000000000 
2022-06-27T00:00:00.000000000 
2022-06-25T13:00:00.000000000 
sell : 
2022-06-30T15:00:00.000000000
DREPUSDT
buy : 
2022-06-30T11:00:00.000000000 
2022-06-28T03:00:00.000000000 
2022-06-27T00:00:00.000000000 
sell : 
2022-06-29T23:00:00.000000000
HOTUSDT
buy : 
2022-06-29T13:00:00.000000000 
2022-06-28T23:00:00.000000000 
2022-06-28T03:00:00.000000000 
sell : 
2022-06-29T20:00:00.000000000
ARUSDT
buy : 
2022-06-28T03:00:00.000000000 
2022-06-27T20:00:00.000000000 
2022-06-27T00:00:00.000000000 
sell : 
2022-06-28T09:00:00.000000000
SYSUSDT
buy : 
2022-06-29T09:00:00.000000000 
2022-06-29T00:00:00.000000000 
2022-06-28T03:00:00.000000000 
sell : 
2022-06-29T22:00:00.000000000
CKBUSDT
buy : 
2022-06-29T07:00:00.000000000 
2022-06-28T03:00:00.000000000 
2022-06-27T17:00:00.000000000 
sell : 
2022-06-29T21:00:00.000000000
CTXCUSDT
buy : 
2022-06-30T09:00:00.000000000 
2022-06-29T17:00:00.000000000 
2022-06-29T07:00:00.000000000 
sell : 
2022-0

In [189]:
data

,Open,High,Low,Close,Volume,ta_rsi
Time,,,,,,
2022-05-28 20:00:00,0.08540000,0.08640000,0.08470000,0.0859,288390.00000000,NaN
2022-05-28 21:00:00,0.08560000,0.08580000,0.08480000,0.0854,116224.00000000,NaN
2022-05-28 22:00:00,0.08540000,0.08600000,0.08480000,0.0848,141515.00000000,NaN
2022-05-28 23:00:00,0.08470000,0.08470000,0.08390000,0.0844,227822.00000000,NaN
2022-05-29 00:00:00,0.08440000,0.08680000,0.08440000,0.0858,145090.00000000,NaN
...,...,...,...,...,...,...
2022-06-30 14:00:00,0.08390000,0.08420000,0.08100000,0.0819,212229.00000000,30.018696
2022-06-30 15:00:00,0.08190000,0.08330000,0.08190000,0.0832,180818.00000000,36.385739
2022-06-30 16:00:00,0.08320000,0.08450000,0.08310000,0.0836,307100.00000000,38.247448


In [187]:



# read from csv files
###########

# n=9
# order=5

# timeframe='2h'
# symbol = 'JASMYUSDT'

# outdir=f"{os.getcwd()}/data/{timeframe}"
# data = pd.read_csv(f"{outdir}/{symbol}_{timeframe}.csv", index_col='Time')


# calc()
# signal_dates(n)

# # the real buy is on the open of the third candle
# print(f'buy : {buy[-1]} \nsell : {sell[-1]}')








In [188]:
# import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

for b in buy:
    fig.add_vline(x=b, line_width=1, line_dash="dash", line_color="green")
for s in sell:
    fig.add_vline(x=s, line_width=1, line_dash="dash", line_color="red")

               
fig.update_layout(template = 'plotly_dark')
fig.layout.xaxis.rangeslider.visible = False


fig.show()